In [203]:
!pip install names
!pip install random-username
!pip install random-password-generator

In [204]:
import names
from random_username.generate import generate_username
from password_generator import PasswordGenerator
import random

In [281]:
def n_rand_digits(n):
    return str(random.randint(1, 9)) + ''.join(["{}".format(random.randint(0, 9)) for num in range(0, n - 1)]) #firt digit isn't 0

def withProbabilityOf(probability):
    return random.random() < probability

def generate_date(startYear, endYear):
    yyyy = random.randint(startYear, endYear)
    mm = random.randint(1,12)
    dd = None
    isLeap = yyyy % 4 == 0 and (yyyy % 100 != 0 or yyyy % 400 == 0)
    if (mm == 1 or mm == 3 or mm == 5 or mm == 7 or mm == 8 or mm == 10 or mm == 12):
        dd = random.randint(1,31)
    elif (mm == 4 or mm == 6 or mm == 9 or mm == 11):
        dd = random.randint(1,30)
    else:
        if(isLeap):
            dd = random.randint(1,29)
        else:
            dd = random.randint(1,28)
    return "{}-{}-{}".format(yyyy,mm,dd)

pwo = PasswordGenerator()
# All properties are optional
pwo.minlen = 10
pwo.maxlen = 30
#pwo.minuchars = 2
pwo.minlchars = 10
#pwo.minnumbers = 1
#pwo.minschars = 1 

In [285]:
def generate_workers(n):
    data = ""
    for i in range(0, n):
        passport = n_rand_digits(10) #PassportNumber
        name = names.get_full_name() #FullName
        bd = generate_date(1940, 2001) #BirthDate
        tax = n_rand_digits(12) #TaxId
        login = generate_username(1)[0]+str(random.randint(0, 99)) #Login
        password = pwo.generate() #Password
        data += "EXEC Add_Worker '{}', '{}', '{}', '{}', '{}', '{}';".format(passport,name,bd,tax,login,password) + "\n"
    return data

def generate_clients(n):
    data = ""
    for i in range(0, n):
        passport = n_rand_digits(10) #PassportNumber
        name = names.get_full_name() #FullName
        bd = generate_date(1940, 2005) #BirthDate
        creatorId = random.randint(1, 100) #AccountCreatorId, the second number here is amount of workers (we pick one of their ids)
        tax = "NULL" #TaxId
        phone = "NULL" #PhoneNumber
        income = "NULL" #IncomePerMonth
        if (withProbabilityOf(0.9)):
            phone = "89" + n_rand_digits(9)
        if (withProbabilityOf(0.4)):
            tax = n_rand_digits(12)
            income = random.randint(10, 200) * 1000 
        data += "EXEC Add_Client '{}', '{}', '{}', '{}', {}, {}, {};".format(passport,name,bd,creatorId,tax,phone,income) + "\n"
    return data

In [286]:
import time
start_time = time.time()

f = open("10kClientsFor100Workers.sql", "w")
f.write(generate_clients(10000))
f.close()

print("--- %s seconds ---" % (time.time() - start_time))

--- 68.75600695610046 seconds ---


In [288]:
 print(generate_clients(1))

EXEC Add_Client '7554418459', 'Patricia Weir', '1990-6-1', '1', 403752527764, 89111640240, 28000;



4388